<a href="https://colab.research.google.com/github/calmrocks/master-machine-learning-engineer/blob/main/MLOps/MLPipelineSagemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Machine Learning Pipeline with Amazon SageMaker

## Overview

This notebook demonstrates how to build an automated ML pipeline using Amazon SageMaker. We'll showcase:
- Automated data processing using SageMaker Processing Jobs
- Model training using SageMaker Training Jobs
- Model deployment using SageMaker Endpoints
- Automated monitoring using Model Monitor
- Automated retraining using SageMaker Pipelines

![ML Pipeline](https://github.com/calmrocks/master-machine-learning-engineer/blob/main/MLOps/Diagrams/MLPipeline.png?raw=1)

## Setup

First, let's set up our SageMaker environment and required dependencies.


In [2]:
!pip install boto3 sagemaker --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.9/404.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.model_monitor import DataCaptureConfig, ModelMonitor

# Setup SageMaker session
session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:11                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:265 in __init__                     │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:293 in _initialize                  │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

## Data Processing Step

Here we define our data processing step using a SageMaker Processing Job.

In [ ]:
from sagemaker.processing import ScriptProcessor

# Create processor for data preprocessing
preprocessor = ScriptProcessor(
    base_job_name='preprocessing-job',
    image_uri='<your-preprocessing-image>',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge'
)

# Define processing step
processing_step = ProcessingStep(
    name="PreprocessData",
    processor=preprocessor,
    inputs=[
        ProcessingInput(
            source='s3://your-bucket/raw-data',
            destination='/opt/ml/processing/input'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='train',
            source='/opt/ml/processing/train'
        ),
        ProcessingOutput(
            output_name='test',
            source='/opt/ml/processing/test'
        )
    ],
    code='preprocess.py'
)

## Training Step

Define the model training step using SageMaker's built-in algorithms or custom containers.

In [ ]:
from sagemaker.estimator import Estimator

# Create estimator
estimator = Estimator(
    image_uri='<your-training-image>',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path='s3://your-bucket/model-output'
)

# Define training step
training_step = TrainingStep(
    name="TrainModel",
    estimator=estimator,
    inputs={
        'training': sagemaker.inputs.TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs['train'].S3Output
        )
    }
)

## Model Evaluation Step

Create a step to evaluate model performance and decide if retraining is needed.

In [ ]:
evaluation_processor = ScriptProcessor(
    base_job_name='evaluation-job',
    image_uri='<your-evaluation-image>',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge'
)

evaluation_step = ProcessingStep(
    name="EvaluateModel",
    processor=evaluation_processor,
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/model'
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs['test'].S3Output,
            destination='/opt/ml/processing/test'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/evaluation'
        )
    ],
    code='evaluate.py'
)

## Model Deployment Step

Configure model deployment with monitoring enabled.

In [ ]:
from sagemaker.model import Model
from sagemaker.workflow.model_step import ModelStep

# Create model
model = Model(
    image_uri='<your-inference-image>',
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    role=role
)

# Define deployment configuration
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri='s3://your-bucket/captured-data'
)

# Model deployment step
deployment_step = ModelStep(
    name="DeployModel",
    step_args=model.deploy(
        initial_instance_count=1,
        instance_type='ml.m5.xlarge',
        data_capture_config=data_capture_config
    )
)

## Automated Monitoring Setup

Configure automated monitoring for the deployed model.

In [ ]:
# Create model monitor
model_monitor = ModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600
)

# Schedule monitoring
model_monitor.create_monitoring_schedule(
    monitor_schedule_name='model-monitor-schedule',
    endpoint_input=deployment_step.properties.EndpointName,
    statistics=baseline_stats,
    constraints=baseline_constraints,
    schedule_cron_expression='cron(0 * ? * * *)'  # Hourly monitoring
)

## Automated Retraining Trigger

Set up automated retraining based on monitoring results.

In [ ]:
from sagemaker.workflow.conditions import ConditionGreaterThanThreshold

# Define condition for retraining
retraining_condition = ConditionGreaterThanThreshold(
    left=evaluation_step.properties.Statistics['DriftMetric'],
    right=0.1  # Threshold for drift
)

# Conditional step for retraining
conditional_training_step = ConditionStep(
    name="CheckDriftAndRetrain",
    conditions=[retraining_condition],
    if_steps=[processing_step, training_step, evaluation_step, deployment_step],
    else_steps=[]
)

## Pipeline Definition

Create and execute the complete pipeline.



# Define pipeline
pipeline = Pipeline(
    name="AutoMLPipeline",
    steps=[
        processing_step,
        training_step,
        evaluation_step,
        conditional_training_step
    ]
)

# Execute pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()

## Pipeline Scheduling

Set up automated pipeline execution.

In [ ]:
import boto3

events_client = boto3.client('events')

# Create CloudWatch Events rule
response = events_client.put_rule(
    Name='MLPipelineSchedule',
    ScheduleExpression='rate(1 day)',  # Run daily
    State='ENABLED'
)

# Add target to trigger pipeline
events_client.put_targets(
    Rule='MLPipelineSchedule',
    Targets=[{
        'Id': 'MLPipelineTarget',
        'Arn': pipeline.arn,
        'RoleArn': role
    }]
)

## Conclusion

This notebook demonstrated how to:

- Create an automated ML pipeline using SageMaker Pipelines
- Set up automated data processing and model training
- Configure model monitoring and drift detection
- Implement automated retraining triggers
- Schedule regular pipeline execution

Key Benefits:

- Fully automated workflow
- Built-in monitoring and retraining
- Scalable and reproducible
- Integrated with AWS services
- Production-ready implementation
